In [1]:
import pandas as pd
import sqlite3


In [4]:
class BookChat:
    def __init__(self):
        self.books = pd.read_csv("./data/books.csv")

    def reply(self, msg: str):
        print(f"BookChat: {msg}")

    def input(self):
        return input("User: ")
    
    def user_msg(self, msg: str):
        print(f"User: {msg}")

In [33]:
class Books(BookChat):
    def __init__(self):
        super().__init__()

    def get_book_details(self, book_title: str):
        return self.books[self.books["title"] == book_title]

    def check_availability(self):
        self.reply(msg="What book are you looking for?")
        user_input = self.input()
        self.user_msg(user_input)
        self.reply(msg="One moment, let me check the inventory.")

        if user_input:
            book = self.books[self.books["title"].str.contains(user_input, case=False)]

            if book.empty:
                self.reply(
                    f"Sorry, it seems that we do not have any book with the title '{user_input}' in stock."
                )

                return []

            else:
                self.reply(
                    f"Yes, '{book.iloc[0]['title']}' by {book.iloc[0]['author']} is currently in stock."
                )

                return book.iloc[0]
        else:
            self.reply("Sorry, I did not catch that. Could you please start again?")
            self.check_stock()

    def check_inventory(self, book):
        self.user_msg("How many copies are available?")

        self.reply("One moment, let me check the inventory.")

        self.reply(f"We currently have {book['quantityInStock']} copies in stock.")

    def check_price(self, book):
        self.user_msg("What is the price of the book?")

        self.reply("Let me find that information for you...")

        self.reply(
            f"{book['title']} by {book['author']} is priced at ${book['price']}."
        )

    def reserve_copy(self, book):
        self.user_msg("Can you reserve a copy for me?")

        self.reply("Sure! Could you please provide your name?")

        name = self.input()
        self.user_msg(name)

        self.reply("Could you please provide an e-mail or phone number?")
        
        contact_info = self.input()
        self.user_msg(contact_info)

        # Update orders table
        
        self.reply(
            f"Thank you, I have reserved a copy of {book['title']} for you, {name}. Please visit the store within the next 48 hours to pick it up."
        )

In [34]:
chatbot = BookChat()
books = Books()

chatbot.reply("Hello, I'm BookChat by Argo Bookstore!")

BookChat: Hello, I'm BookChat by Argo Bookstore!


### Scenario 1

#### Scene 1.1

In [21]:
# User: "Do you have '1984' by George Orwell in stock?"
# Chatbot: "Let me check that for you."
# The chatbot would then use the Pandas API to search the inventory database for the book.
# Chatbot: "Yes, '1984' by George Orwell is currently in stock."

In [35]:
bookSearched = books.check_availability()

BookChat: What book are you looking for?
User: monte cristo
BookChat: One moment, let me check the inventory.
BookChat: Yes, 'The Count of Monte Cristo'' by 'Alexandre Dumas' is currently in stock.


In [39]:
books.check_inventory(bookSearched)
books.check_price(bookSearched)
books.reserve_copy(bookSearched)

User: How many copies are available?
BookChat: One moment, let me check the inventory.
BookChat: We currently have 20 copies in stock.
User: What is the price of the book?
BookChat: Let me find that information for you...
BookChat: The Count of Monte Cristo by Alexandre Dumas is priced at $10.99.
User: Can you reserve a copy for me?
BookChat: Sure! Could you please provide your name?
User: 
BookChat: Could you please provide an e-mail or phone number?
User: 
BookChat: Thank you, I have reserved a copy of The Count of Monte Cristo for you, . Please visit the store within the next 48 hours to pick it up.
